In [11]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Dropout, LSTM

import tensorflow as tf


import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [12]:
df_tele = pd.read_csv("Dados/TelemetryData_470723873226893378.csv")
df_session = pd.read_csv ("Dados/SessionData_470723873226893378.csv")


C:\Users\andre\AppData\Local\Temp\ipykernel_19496\1293583313.py:1: DtypeWarning: Columns (52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tele = pd.read_csv("Dados/TelemetryData_470723873226893378.csv")


In [13]:
df_tele.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071280 entries, 0 to 1071279
Data columns (total 56 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   sessionTime              1071280 non-null  float64
 1   frameIdentifier          1071280 non-null  int64  
 2   pilot_index              1071280 non-null  int64  
 3   worldPositionX           1071280 non-null  float64
 4   worldPositionY           1071280 non-null  float64
 5   worldPositionZ           1071280 non-null  float64
 6   worldVelocityX           1071280 non-null  float64
 7   worldVelocityY           1071280 non-null  float64
 8   worldVelocityZ           1071280 non-null  float64
 9   worldForwardDirX         1071280 non-null  int64  
 10  worldForwardDirY         1071280 non-null  int64  
 11  worldForwardDirZ         1071280 non-null  int64  
 12  worldRightDirX           1071280 non-null  int64  
 13  worldRightDirY           1071280 non-null 

In [14]:
df_tyre = df_tele[[
    "currentLapNum",
    "tyresSurfaceTemperature",
    "tyresInnerTemperature",
    "tyresPressure",
    "fuelInTank",
    "tyresWear",
    "actualTyreCompound",
    "tyresDamage"
]]

In [15]:
def replace_with_mean(value):
    # Verificar se o valor é uma string
    if isinstance(value, str):
        # Dividir a string em quatro valores
        values = value.split('/')
        # Converter os valores em números de ponto flutuante
        values = [float(v) for v in values]
        # Calcular a média dos valores
        mean_value = sum(values) / len(values)
        # Retornar a média como uma string
        return str(mean_value)
    else:
        # Se o valor não for uma string, retorná-lo sem modificação
        return value

# Aplicar a função personalizada 
df_tyre['tyresSurfaceTemperature'] = df_tyre['tyresSurfaceTemperature'].apply(replace_with_mean)
df_tyre['tyresInnerTemperature'] = df_tyre['tyresInnerTemperature'].apply(replace_with_mean)
df_tyre['tyresPressure'] = df_tyre['tyresPressure'].apply(replace_with_mean)
df_tyre['tyresWear'] = df_tyre['tyresWear'].apply(replace_with_mean)
df_tyre['tyresDamage'] = df_tyre['tyresDamage'].apply(replace_with_mean)

C:\Users\andre\AppData\Local\Temp\ipykernel_19496\154789895.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tyre['tyresSurfaceTemperature'] = df_tyre['tyresSurfaceTemperature'].apply(replace_with_mean)
C:\Users\andre\AppData\Local\Temp\ipykernel_19496\154789895.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tyre['tyresInnerTemperature'] = df_tyre['tyresInnerTemperature'].apply(replace_with_mean)
C:\Users\andre\AppData\Local\Temp\ipykernel_19496\154789895.py:19: SettingWithCopyWarning: 
A va

In [16]:
df_tyre = pd.concat([df_tyre, pd.get_dummies(df_tyre['actualTyreCompound'], prefix='tyre')], axis=1)
df_tyre = df_tyre.drop('actualTyreCompound', axis=1)
df_tyre['tyresWear'] = pd.to_numeric(df_tyre['tyresWear'], errors='coerce')
df_tyre['tyresSurfaceTemperature'] = pd.to_numeric(df_tyre['tyresSurfaceTemperature'], errors='coerce')
df_tyre['tyresInnerTemperature'] = pd.to_numeric(df_tyre['tyresInnerTemperature'], errors='coerce')
df_tyre['tyresPressure'] = pd.to_numeric(df_tyre['tyresPressure'], errors='coerce')


data = df_tyre.dropna(subset=['tyresSurfaceTemperature'])
data = df_tyre.dropna(subset=['tyresInnerTemperature'])
data = df_tyre.dropna(subset=['tyresPressure'])

In [17]:
scaler = MinMaxScaler()
df_tyre['fuelInTank'] = scaler.fit_transform(df_tyre['fuelInTank'].values.reshape(-1, 1))
df_tyre['currentLapNum'] = scaler.fit_transform(df_tyre['currentLapNum'].values.reshape(-1, 1))

df_tyre['tyresSurfaceTemperature'] = scaler.fit_transform(df_tyre[['tyresSurfaceTemperature']])
df_tyre['tyresInnerTemperature'] = scaler.fit_transform(df_tyre[['tyresInnerTemperature']])
df_tyre['tyresPressure'] = scaler.fit_transform(df_tyre[['tyresPressure']])
df_tyre['tyresWear'] = scaler.fit_transform(df_tyre[['tyresWear']])
df_tyre['tyresDamage'] = scaler.fit_transform(df_tyre[['tyresDamage']])

df_tyre['tyresWear'] = pd.to_numeric(df_tyre['tyresWear'], errors='coerce')

df_tyre

currentLapNum  tyresSurfaceTemperature  tyresInnerTemperature  \
0             0.000000                 0.475472               0.718232   
1             0.000000                 0.475472               0.718232   
2             0.000000                 0.475472               0.718232   
3             0.000000                 0.475472               0.718232   
4             0.000000                 0.475472               0.718232   
...                ...                      ...                    ...   
1071275       0.944444                 0.494340               0.790055   
1071276       0.972222                 0.483019               0.779006   
1071277       0.944444                 0.547170               0.762431   
1071278       0.555556                 0.000000               0.022099   
1071279       1.000000                 0.464151               0.745856   

         tyresPressure  fuelInTank  tyresWear  tyresDamage  tyre_hard  \
0             1.000000    0.940703   0.000000     0.000000          0   
1             1.000000    0.950742   0.000000     0.000000          0   
2             1.000000    0.957484   0.000000     0.000000          0   
3             1.000000    0.915815   0.000000     0.000000          0   
4             1.000000    0.954756   0.000000     0.000000          0   
...                ...         ...        ...          ...        ...   
1071275       0.918251    0.030969   0.921348     0.921348          1   
1071276       0.918251    0.018788   0.797753     0.797753          1   
1071277       0.918251    0.023379   0.865169     0.865169          1   
1071278       0.918251    0.447419   0.269663     0.269663          1   
1071279       0.091866    0.016878   0.898876     0.898876          1   

         tyre_medium  tyre_soft  
0                  0          1  
1                  0          1  
2                  0          1  
3                  1          0  
4                  0          1  
...              ...        ...  
1071275            0          0  
1071276            0          0  
1071277            0          0  
1071278            0          0  
1071279            0          0  

[1071280 rows x 10 columns]

In [18]:
X = df_tyre

df_tele["pitStatus"] = df_tele["pitStatus"].replace('pitting',1)
df_tele["pitStatus"] = df_tele["pitStatus"].replace('in pit area',1)
y = df_tele["pitStatus"]

y.fillna(0,inplace=True)



In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.nan_to_num(X_train, nan=0.0)
X_test = np.nan_to_num(X_test,nan=0.0)
y_train = np.nan_to_num(y_train, nan=0.0)
y_test = np.nan_to_num(y_test,nan=0.0)

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
lstm_units = 64
dropout_rate = 0.5
learning_rate = 0.001
epochs = 10
batch_size = 32

# Construindo o
model = Sequential()
model.add(Dense(lstm_units, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))

# Compilando o modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Treinando o modelo
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

# Avaliando o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

Epoch 1/10
21426/21426 [==============================] - 85s 4ms/step - loss: 0.0279 - accuracy: 0.9936 - val_loss: 0.0177 - val_accuracy: 0.9949
Epoch 2/10
21426/21426 [==============================] - 81s 4ms/step - loss: 0.0196 - accuracy: 0.9945 - val_loss: 0.0151 - val_accuracy: 0.9955
Epoch 3/10
21426/21426 [==============================] - 80s 4ms/step - loss: 0.0178 - accuracy: 0.9947 - val_loss: 0.0157 - val_accuracy: 0.9951
Epoch 4/10
21426/21426 [==============================] - 91s 4ms/step - loss: 0.0166 - accuracy: 0.9948 - val_loss: 0.0139 - val_accuracy: 0.9955
Epoch 5/10
21426/21426 [==============================] - 95s 4ms/step - loss: 0.0161 - accuracy: 0.9949 - val_loss: 0.0130 - val_accuracy: 0.9956
Epoch 6/10
21426/21426 [==============================] - 92s 4ms/step - loss: 0.0156 - accuracy: 0.9950 - val_loss: 0.0125 - val_accuracy: 0.9955
Epoch 7/10
21426/21426 [==============================] - 86s 4ms/step - loss: 0.0152 - accuracy: 0.9951 - val_loss: 0

In [20]:
y_pred = model.predict(X_test)



recomended_pitstop = np.where(y_pred )[0]

recomended_pitstop

6696/6696 [==============================] - 15s 2ms/step


array([     0,      1,      2, ..., 214253, 214254, 214255], dtype=int64)

In [21]:
df_tele.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071280 entries, 0 to 1071279
Data columns (total 56 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   sessionTime              1071280 non-null  float64
 1   frameIdentifier          1071280 non-null  int64  
 2   pilot_index              1071280 non-null  int64  
 3   worldPositionX           1071280 non-null  float64
 4   worldPositionY           1071280 non-null  float64
 5   worldPositionZ           1071280 non-null  float64
 6   worldVelocityX           1071280 non-null  float64
 7   worldVelocityY           1071280 non-null  float64
 8   worldVelocityZ           1071280 non-null  float64
 9   worldForwardDirX         1071280 non-null  int64  
 10  worldForwardDirY         1071280 non-null  int64  
 11  worldForwardDirZ         1071280 non-null  int64  
 12  worldRightDirX           1071280 non-null  int64  
 13  worldRightDirY           1071280 non-null 

In [22]:
recommended_laps = df_tele.loc[recomended_pitstop, 'currentLapNum']
recommended_laps


0         1
1         1
2         1
3         1
4         1
         ..
214251    8
214252    7
214253    7
214254    7
214255    7
Name: currentLapNum, Length: 214256, dtype: int64